In [10]:
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

In [26]:
#american clean
start = "C:/Users/talno/microbiom_project/american_gut/"
data_americans = pd.read_csv(start + "feature_table_gtdb.tsv", sep='\t')
data_americans.drop('ID',inplace=True, axis=1)
data_americans["taxon1"] = data_americans['taxon'].str.split(';', expand=True)[5]
data_americans["taxon2"] = data_americans['taxon'].str.split(';', expand=True)[4]
data_americans = data_americans.groupby(['taxon2', 'taxon1']).sum()
data_americans = data_americans/data_americans.sum(axis=0)
data_americans.fillna(0,inplace=True)
data_americans = data_americans.T

C:\Users\talno\AppData\Local\Temp\ipykernel_35068\4078193045.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  data_americans = data_americans.groupby(['taxon2', 'taxon1']).sum()


In [45]:
ibd_huttenhower = pd.read_csv("C:/Users/talno/microbiom_project/ibd_huttenhower_results/RDP/ibd_huttenhower.otu_table.100.denovo.rdp_assigned",sep='\t')
meta_huttenhower = pd.read_csv("C:/Users/talno/microbiom_project/ibd_huttenhower_results/ibd_huttenhower.metadata.txt",sep='\t')
# arrange the otu with disease state and where from and take just what appears in the meta data
def clean(meta,ibd,name):
    
    otu = ibd.copy()
    otu["taxon_g"] = otu["Unnamed: 0"].str.split(';', expand=True)[5]
    otu["taxon_f"] = otu["Unnamed: 0"].str.split(';', expand=True)[4]
    otu = otu.groupby(["taxon_f", "taxon_g"]).sum()
    try:
        otu.drop(('f__', 'g__'), inplace=True)
    except Exception:
        print("no g__, f__")
    otu = otu /otu.sum(axis = 0)
    otu.fillna(0,inplace=True)
    otu = otu.T
    meta.set_index("#SampleID", inplace=True)
    keepsmpls = [i for i in otu.index if i in meta.index]
    otu = otu.loc[keepsmpls]
    meta = meta.loc[keepsmpls]
    otu = otu.join(meta.DiseaseState)
    otu.loc[:, "DiseaseState"] = otu.loc[:, "DiseaseState"].map(
        {"ASD": "D", "nonCDI": "H", "nonIBD": "H", "CD": "D", "UC": "D", "CDI": "D", "H": "H", "postFMT_CDI": "H",
         "ASD": "D", 'HIV': "D", 'EDD': "D", "CRC": 'D', 'OB': "D", 'PAR': "D", "OW": "D", "CIRR": "D", "MHE": "D",
         "nonCRC": "H", "RA": "D", "T1D": "D", "CRC": "D"})
    otu["From"] = name
    # print(otu.shape, name,otu[otu["DiseaseState"] == "H"].shape,otu[otu["DiseaseState"] == "D"].shape)
    return otu

otu = clean(meta_huttenhower, ibd_huttenhower, "hutt")

C:\Users\talno\AppData\Local\Temp\ipykernel_35068\2440006211.py:9: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  otu = otu.groupby(["taxon_f", "taxon_g"]).sum()
C:\Users\talno\AppData\Local\Temp\ipykernel_35068\2440006211.py:21: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left, 1 on the right)
  otu = otu.join(meta.DiseaseState)


In [48]:
otu.sum(axis=1)

C:\Users\talno\AppData\Local\Temp\ipykernel_35068\2035582637.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  otu.sum(axis=1)


100225    1.0
100229    1.0
7007      1.0
7094C     1.0
7111      1.0
         ... 
7016      1.0
7267      1.0
100166    1.0
7160      1.0
7251      1.0
Length: 228, dtype: float64

In [27]:
data_americans.columns = data_americans.columns.get_level_values(1)

In [29]:
data_americans.sum(axis=0)

taxon1
g__                       188.249050
g__                         0.016275
g__Roseomonas               0.041614
g__                         5.195571
g__Acidaminococcus          5.019443
                             ...    
g__                         0.043099
g__Luteimonas               0.018965
g__Pseudoxanthomonas_A      2.524441
g__Stenotrophomonas        48.797322
g__Xanthomonas              1.200280
Length: 752, dtype: float64

In [5]:
data = pd.read_csv("C:/Users/talno/microbiom_project/microbiome_dysbiosis/microbiome_dysbiosis/clean_concat_microbiomeHD")

In [7]:
data.set_index("Unnamed: 0", inplace=True)

In [9]:
data.columns

Index(['g__Abiotrophia', 'g__Acetanaerobacterium', 'g__Acetobacterium',
       'g__Acidaminococcus', 'g__Acidovorax', 'g__Acinetobacter',
       'g__Actinobacillus', 'g__Actinomyces', 'g__Aeromonas',
       'g__Aggregatibacter',
       ...
       'g__Pseudacidovorax', 'g__Ruminobacter', 'g__Shimwellia',
       'g__Sphingobacterium', 'g__Spirochaeta', 'g__Wautersiella',
       'g__Blastomonas', 'g__Burkholderia', 'g__Gp1', 'g__Streptobacillus'],
      dtype='object', length=429)